# Performing NCD analysis: burden of disease

## Step-by-step approach:

- Import NCD data
- Import SVRS data and check available RMNCAH variables
- Identify the key variables that could be used as RMNCAH from SVRS
- 
    

In [496]:
import os
import re
import sys
import glob
import tzlocal
from fuzzywuzzy import fuzz
import numpy as np
import pandas as pd
from scipy import io
from dbfread import DBF
from collections import Counter
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
from rpy2.robjects import packages
from rpy2.robjects import pandas2ri
from simpledbf import Dbf5
from dbfread import DBF

def read_dbf(file):
    tmp = DBF(file)
    tmp = pd.DataFrame(iter(tmp))
    return tmp

def read_sav(file):
    foreign = packages.importr('foreign')
    pandas2ri.activate()
    df = foreign.read_spss(file, reencode=False)
    return pd.DataFrame(dict(zip(df.names, map(list,list(df)))))

def read_dta(file):
    itr = pd.read_stata(file, iterator=True)
    variables = list(itr.variable_labels().values())
    data = pd.read_stata(os.path.join(file))
    original_variables = data.columns
    data.columns = variables
    return data, original_variables
def match_districts(ref_df, ref_match, input_df, idvar):
    out = pd.DataFrame()
    print(input_df.columns)
    for key, code in enumerate(input_df[idvar]):
        code_match = {}
        code_match['FuzzRatio'] = [fuzz.ratio(ref_code, code)  for ref_code in ref_df[ref_match]]
        code_match['Geo'] = code
        code_match['DivisionCode'] = [value[0] for value in ref_df.values]
        code_match['DivisionName'] = [value[1]  for value in ref_df.values]
        code_match['DistrictCode'] = [value[2]  for value in ref_df.values]
        code_match['DistrictName'] = [value[3]  for value in ref_df.values]
        code_match = pd.DataFrame.from_dict(code_match)
        out = out.append(code_match.sort_values('FuzzRatio', ascending=False).iloc[0,:])
    for var in list(out.columns):
        if out[var].dtype.kind == 'f':
            out[var] = out[var].astype(int)
            out[var] = out[var].astype(str)
            code_length = max([len(char) for char in out[var]])
            out[var] = out[var].str.pad(width=code_length, side='left', fillchar='0') 
    return out

## Reading data

In [345]:
SVRS2012 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_12/tafsil-2p.dbf'
SVRS2014 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_14/tafsil2P_12-06-2015.sav'
SVRS2015 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_15/tafsil-2p.dbf'
SVRS2017 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_17/tafsl-2p.dta'

In [346]:
svrs2012 = read_dbf(SVRS2012)
SAMPLESIZE2012 = svrs2012.shape[0]
svrs2014 = read_sav(SVRS2014)
SAMPLESIZE2014 = svrs2014.shape[0]
svrs2015 = read_dbf(SVRS2015)
SAMPLESIZE2015 = svrs2015.shape[0]
svrs2017, original_variables = read_dta(SVRS2017)
SAMPLESIZE2017 = svrs2017.shape[0]
print(SAMPLESIZE2012)
print(SAMPLESIZE2014)
print(SAMPLESIZE2015)
print(SAMPLESIZE2017)

1116845
692412
939530
1252581


In [268]:
SVRS2012_3 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_12/tafsil-3.dbf'
SVRS2012_9 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_12/tafsil-9.dbf'
SVRS2014_3 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_14/tafsil3_12-06-2015.sav'
SVRS2014_9 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_14/tafsil9_12-06-2015.sav'
SVRS2015_3 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_15/tafsil-3.dbf'
SVRS2015_9 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_15/tafsil-9.dbf'
SVRS2017_3 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_17/tafsil-3.dta'
SVRS2017_9 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/svrs/SVRS_17/tafsil-9.dta'
NCD = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/ncd/prevalence_all.csv'
variables = ['Education', 'Age', 'Urban/Rural', 'Gender']

In [483]:
svrs2014.head()

,HH1,div,zila,upz,union,mauza,RMO,area,HH2,Q9,...,Q16,Q17,Q18,Q19,Q20,Q21,religion,eduhh,stratum,hhweight
0,757.0,Barisal,Barguna,9.0,7.0,250.0,Municipality,Urban,1.0,1.0,...,13,No,Not applicable,Yes,Institutional school,16,Hindu,Above secondary,Barguna Urban,0.31293
1,757.0,Barisal,Barguna,9.0,7.0,250.0,Municipality,Urban,1.0,2.0,...,9,No,Not applicable,Yes,Institutional school,25,Hindu,Above secondary,Barguna Urban,0.31293
2,757.0,Barisal,Barguna,9.0,7.0,250.0,Municipality,Urban,1.0,3.0,...,5,Yes,No,Yes,Institutional school,24,Hindu,Above secondary,Barguna Urban,0.31293
3,757.0,Barisal,Barguna,9.0,7.0,250.0,Municipality,Urban,1.0,4.0,...,10,No,Not applicable,Yes,Institutional school,Other,Hindu,Above secondary,Barguna Urban,0.31293
4,757.0,Barisal,Barguna,9.0,7.0,250.0,Municipality,Urban,1.0,5.0,...,11,No,Not applicable,Yes,Institutional school,Other,Hindu,Above secondary,Barguna Urban,0.31293


In [269]:
svrs2012_3 = read_dbf(SVRS2012_3)
svrs2012_9 = read_dbf(SVRS2012_9)
svrs2014_3 = read_sav(SVRS2014_3)
svrs2014_9 = read_sav(SVRS2014_9)
svrs2015_3 = read_dbf(SVRS2015_3)
svrs2015_9 = read_dbf(SVRS2015_9)
svrs2017_3 = read_dta(SVRS2017_3)
svrs2017_9 = read_dta(SVRS2017_9)

In [300]:
print(svrs2012_3['Q_9'].value_counts()/SAMPLESIZE2012*1000)
print(svrs2014_3['C9'].value_counts()/SAMPLESIZE2014*1000)
print(svrs2015_3['Q_9'].value_counts()/SAMPLESIZE2015*1000)
print(svrs2017_3[0][' Live Birth '].value_counts()/SAMPLESIZE2017*1000)

1    18.707162
2     0.233694
Name: Q_9, dtype: float64
Live birth     18.837051
Still birth     0.187749
Name: C9, dtype: float64
1    18.557151
2     0.255447
Name: Q_9, dtype: float64
1    18.173675
2     0.352073
Name:  Live Birth , dtype: float64


In [270]:
ncd_data = pd.read_csv(NCD)
ncd_data.head()

,DIVN_ZILA,Hyperglycaemia count_d2012,Total count_d2012,Hyperglycaemia prevalence_d2012,Hyperglycaemia count_d2014,Total count_d2014,Hyperglycaemia prevalence_d2014,Hyperglycaemia count_d2015,Total count_d2015,Hyperglycaemia prevalence_d2015,...,Hypertension prevalence_h2012,Hypertension count_h2014,Total count_h2014,Hypertension prevalence_h2014,Hypertension count_h2015,Total count_h2015,Hypertension prevalence_h2015,Hypertension count_h2017,Total count_h2017,Hypertension prevalence_h2017
0,Bagerhat,1010,13369,7.55,450,5440,8.27,834,9270,9.00,...,11.51,1135,5440,20.86,1931,9270,20.83,2569,12012,21.39
1,Bandarban,199,11210,1.78,41,1948,2.10,91,3628,2.51,...,8.68,218,1948,11.19,455,3628,12.54,492,3295,14.93
2,Barguna,764,13868,5.51,337,4528,7.44,583,7773,7.50,...,12.46,926,4528,20.45,1486,7773,19.12,1961,9631,20.36
3,Barisal,1468,19779,7.42,2035,28069,7.25,2636,45334,5.81,...,11.94,4472,28069,15.93,6520,45334,14.38,3993,21077,18.94
4,Bhola,636,16751,3.80,443,8685,5.10,1119,15993,7.00,...,9.57,1333,8685,15.35,2670,15993,16.69,2530,15341,16.49


## CBR 2012

In [571]:
sam2012 = svrs2012[['DIVN', 'ZILA']]
sam2012['DIVN_ZILA'] = sam2012['DIVN'].str.cat(sam2012['ZILA'])
sam2012 = sam2012[['DIVN_ZILA', 'DIVN']].groupby('DIVN_ZILA').count()
sam2012 = sam2012.reset_index()
cbr2012 = svrs2012_3[['DIVN', 'ZILA', 'Q_9']]
cbr2012['DIVN_ZILA'] = cbr2012['DIVN'].str.cat(cbr2012['ZILA'])
cbr2012 = cbr2012[['DIVN_ZILA', 'Q_9']].groupby('DIVN_ZILA').count()
cbr2012 = cbr2012.reset_index()
cbr2012 = sam2012.merge(cbr2012, how='left', on='DIVN_ZILA')
cbr2012.columns = ['ZILA','SampleSize', 'LiveBirths']
cbr2012.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ZILA,SampleSize,LiveBirths
0,1004,13871,285
1,1006,19779,383
2,1009,16772,305
3,1042,12639,231
4,1078,16259,315


## CBR 2014

In [572]:
sam2014 = svrs2014[['zila', 'upz']].groupby('zila').count()
sam2014.columns = ['DIVN_ZILA']
sam2014 = sam2014.reset_index()
cbr2014 = svrs2014_3[['zila', 'C9']]
cbr2014 = cbr2014[['zila', 'C9']].groupby('zila').count()
cbr2014 = cbr2014.reset_index()
cbr2014 = sam2014.merge(cbr2014, how='left', on='zila')
cbr2014.columns = ['ZILA','SampleSize', 'LiveBirths']
cbr2014.head()

,ZILA,SampleSize,LiveBirths
0,Bagerhat,6126,107
1,Bandarban,2208,29
2,Barguna,4977,94
3,Barisal,31252,530
4,Bhola,9684,211


## CBR 2015

In [573]:
sam2015 = svrs2015[['ZILA', 'UPZA']].groupby('ZILA').count()
sam2015.columns = ['SampleSize']
sam2015 = sam2015.reset_index()
cbr2015 = svrs2015_3[['ZILA', 'Q_9']]
cbr2015 = cbr2015[['ZILA', 'Q_9']].groupby('ZILA').count()
cbr2015 = cbr2015.reset_index()
cbr2015 = sam2015.merge(cbr2015, how='left', on='ZILA')
cbr2015.columns = ['ZILA','SampleSize','LiveBirths']
cbr2015.head()

,ZILA,SampleSize,LiveBirths
0,01,9274,174
1,03,3629,71
2,04,7773,127
3,06,45404,747
4,09,15999,362


## CBR 2017

In [574]:
sam2017 = svrs2017[['Zila code', ' Upazila code']].groupby('Zila code').count()
sam2017 = sam2017.reset_index()
sam2017['Zila code'] = sam2017['Zila code'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
sam2017.columns = ['ZILA', 'SampleSize']
cbr2017 = svrs2017_3[0][[' Zila Code', ' Live Birth ']]
cbr2017[' Zila Code'] = cbr2017[' Zila Code'].astype(str).str.pad(width=2, side='left', fillchar='0')
cbr2017.columns = ['ZILA', 'LiveBirths']
cbr2017 = cbr2017.groupby('ZILA').count()
cbr2017 = cbr2017.reset_index()
cbr2017 = sam2017.merge(cbr2017, how='left', on='ZILA')
print(cbr2017.shape)
cbr2017.head()

(64, 3)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ZILA,SampleSize,LiveBirths
0,01,12526,229
1,03,4646,91
2,04,10230,171
3,06,55321,1080
4,09,22253,287


## Geos

In [575]:
geos = pd.read_excel('/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/geo_files/geos_svrs.xlsx')
geos = geos[['dv','Div_Name','zl','Zila_Name','Zila_Name1', 'Zila_Name2']]
print(geos.shape)
geos = geos.drop_duplicates()
geos['zl'] = geos['zl'].astype(str).str.pad(width=2, side='left', fillchar='0')
geos['ZILA'] = geos['dv'].astype(str).str.cat(geos['zl'])
print(geos.shape)
geos.head()


(2012, 6)
(65, 7)


,dv,Div_Name,zl,Zila_Name,Zila_Name1,Zila_Name2,ZILA
0,10,Barisal,04,Barguna,Barguna,Barguna,1004
17,10,Barisal,06,Barisal,Barisal,Barishal,1006
111,10,Barisal,09,Bhola,Bhola,Bhola,1009
143,10,Barisal,42,Jhalokati,Jhalokati,Jhalokati,1042
157,10,Barisal,78,Patuakhali,Patuakhali,Patuakhali,1078


## Merging data

In [576]:
print(cbr2012.shape)
cbr2012o = cbr2012.merge(geos, how='left',on='ZILA')
display(cbr2012o.head())
print(cbr2012o.shape)
print("#"*30)
print(cbr2014.shape)
df = match_districts(ref_df=geos, ref_match='Zila_Name2', input_df=cbr2014, idvar='ZILA')     
cbr2014o = cbr2014.merge(df, how='left',left_on='ZILA', right_on='Geo')
print(cbr2014o.shape)
display(cbr2014o.head())
print("#"*30)
print(cbr2015.shape)
cbr2015o = cbr2015.merge(geos, how='left',left_on='ZILA', right_on='zl')
print(cbr2015o.shape)
display(cbr2014o.head())
print("#"*30)
print(cbr2017.shape)
cbr2017o = cbr2017.merge(geos, how='left',left_on='ZILA', right_on='zl')
print(cbr2017o.shape)
display(cbr2017o.head())
print("#"*30)

(64, 3)


,ZILA,SampleSize,LiveBirths,dv,Div_Name,zl,Zila_Name,Zila_Name1,Zila_Name2
0,1004,13871,285,10,Barisal,04,Barguna,Barguna,Barguna
1,1006,19779,383,10,Barisal,06,Barisal,Barisal,Barishal
2,1009,16772,305,10,Barisal,09,Bhola,Bhola,Bhola
3,1042,12639,231,10,Barisal,42,Jhalokati,Jhalokati,Jhalokati
4,1078,16259,315,10,Barisal,78,Patuakhali,Patuakhali,Patuakhali


(65, 9)
##############################
(64, 3)
Index(['ZILA', 'SampleSize', 'LiveBirths'], dtype='object')
(64, 9)


,ZILA,SampleSize,LiveBirths,DistrictCode,DistrictName,DivisionCode,DivisionName,FuzzRatio,Geo
0,Bagerhat,6126,107,01,Bagerhat,40,Khulna,100,Bagerhat
1,Bandarban,2208,29,03,Bandarban,20,Chittagong,100,Bandarban
2,Barguna,4977,94,04,Barguna,10,Barisal,100,Barguna
3,Barisal,31252,530,06,Barisal,10,Barisal,093,Barisal
4,Bhola,9684,211,09,Bhola,10,Barisal,100,Bhola


##############################
(64, 3)
(65, 10)


,ZILA,SampleSize,LiveBirths,DistrictCode,DistrictName,DivisionCode,DivisionName,FuzzRatio,Geo
0,Bagerhat,6126,107,01,Bagerhat,40,Khulna,100,Bagerhat
1,Bandarban,2208,29,03,Bandarban,20,Chittagong,100,Bandarban
2,Barguna,4977,94,04,Barguna,10,Barisal,100,Barguna
3,Barisal,31252,530,06,Barisal,10,Barisal,093,Barisal
4,Bhola,9684,211,09,Bhola,10,Barisal,100,Bhola


##############################
(64, 3)
(65, 10)


,ZILA_x,SampleSize,LiveBirths,dv,Div_Name,zl,Zila_Name,Zila_Name1,Zila_Name2,ZILA_y
0,01,12526,229,40,Khulna,01,Bagerhat,Bagerhat,Bagerhat,4001
1,03,4646,91,20,Chittagong,03,Bandarban,Bandarban,Bandarban,2003
2,04,10230,171,10,Barisal,04,Barguna,Barguna,Barguna,1004
3,06,55321,1080,10,Barisal,06,Barisal,Barisal,Barishal,1006
4,09,22253,287,10,Barisal,09,Bhola,Bhola,Bhola,1009


##############################


In [577]:
cbr2017o.head()

,ZILA_x,SampleSize,LiveBirths,dv,Div_Name,zl,Zila_Name,Zila_Name1,Zila_Name2,ZILA_y
0,01,12526,229,40,Khulna,01,Bagerhat,Bagerhat,Bagerhat,4001
1,03,4646,91,20,Chittagong,03,Bandarban,Bandarban,Bandarban,2003
2,04,10230,171,10,Barisal,04,Barguna,Barguna,Barguna,1004
3,06,55321,1080,10,Barisal,06,Barisal,Barisal,Barishal,1006
4,09,22253,287,10,Barisal,09,Bhola,Bhola,Bhola,1009


In [578]:
cbr2012o['CBR'] = cbr2012o['LiveBirths']/cbr2012o['SampleSize']*1000
cbr2014o['CBR'] = cbr2014o['LiveBirths']/cbr2014o['SampleSize']*1000
cbr2014o['ZILA_'] = cbr2014o['DivisionCode'].str.cat(cbr2014o['DistrictCode'])
cbr2015o['CBR'] = cbr2015o['LiveBirths']/cbr2015o['SampleSize']*1000
cbr2017o['CBR'] = cbr2017o['LiveBirths']/cbr2017o['SampleSize']*1000

In [579]:
cbr2012o = cbr2012o[['ZILA','SampleSize','LiveBirths', 'CBR']]
cbr2012o.columns = ['ZILA','SampleSize 2012','LiveBirths 2012', 'CBR 2012']
cbr2014o = cbr2014o[['ZILA_','SampleSize','LiveBirths', 'CBR']]
cbr2014o.columns = ['ZILA','SampleSize 2014','LiveBirths 2014', 'CBR 2014']
cbr2015o = cbr2015o[['ZILA_y','SampleSize','LiveBirths', 'CBR']]
cbr2015o.columns = ['ZILA','SampleSize 2015','LiveBirths 2015', 'CBR 2015']
cbr2017o = cbr2017o[['ZILA_y','SampleSize','LiveBirths', 'CBR']]
cbr2017o.columns = ['ZILA','SampleSize 2017','LiveBirths 2017', 'CBR 2017']

In [580]:
display(cbr2012o.head())
display(cbr2014o.head())
display(cbr2015o.head())
display(cbr2017o.head())

,ZILA,SampleSize 2012,LiveBirths 2012,CBR 2012
0,1004,13871,285,20.546464
1,1006,19779,383,19.363972
2,1009,16772,305,18.185070
3,1042,12639,231,18.276762
4,1078,16259,315,19.373885


,ZILA,SampleSize 2014,LiveBirths 2014,CBR 2014
0,4001,6126,107,17.466536
1,2003,2208,29,13.134058
2,1004,4977,94,18.886880
3,1006,31252,530,16.958915
4,1009,9684,211,21.788517


,ZILA,SampleSize 2015,LiveBirths 2015,CBR 2015
0,4001,9274,174,18.762131
1,2003,3629,71,19.564618
2,1004,7773,127,16.338608
3,1006,45404,747,16.452295
4,1009,15999,362,22.626414


,ZILA,SampleSize 2017,LiveBirths 2017,CBR 2017
0,4001,12526,229,18.281973
1,2003,4646,91,19.586741
2,1004,10230,171,16.715543
3,1006,55321,1080,19.522424
4,1009,22253,287,12.897137


In [587]:
cbr2014o.shape

(64, 4)

In [600]:
cbr = cbr2014o.merge(cbr2015o, how='left', on='ZILA')
cbr = cbr.merge(cbr2012o, how='left', on='ZILA')
cbr = cbr.merge(cbr2017o, how='left', on='ZILA')
cbr = cbr.sort_values('ZILA')
cbr = cbr.drop_duplicates()
cbr.to_csv('/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/ncd/crude_birth_rate.csv', index=False, index_label=False)